# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [2]:
import napari
from macrohet import dataio, tile, visualise
# import btrack
# print(btrack.__version__)
import os, glob

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [16]:
expt_ID = 'ND0001'
base_dir = f'/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/{expt_ID}/'#f'/mnt/DATA/macrohet/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


FileNotFoundError: [Errno 2] No such file or directory: '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/ND0001/acquisition/Images/Index.idx.xml'

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [14]:
os.path.join(base_dir, 'acquisition/Assaylayout/*.xml')

'/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/ND0002/acquisition/Assaylayout/*.xml'

In [15]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

### Define row and column of choice

In [6]:
row = 1
column = 3

### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [7]:
subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [175]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             subset_field_IDs=['16', '17',  '20', '21'], 
                             n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

CPU times: user 119 ms, sys: 3.34 ms, total: 123 ms
Wall time: 217 ms


In [176]:
images

dask.array<max-aggregate, shape=(1, 3, 2052, 2052), dtype=uint16, chunksize=(1, 1, 1080, 1080), chunktype=numpy.ndarray>

In [177]:
%%time
images = images.compute().compute()

CPU times: user 46.7 s, sys: 9.25 s, total: 56 s
Wall time: 13.7 s


In [178]:
images.shape

(1, 3, 2052, 2052)

In [179]:
viewer = napari.Viewer()

viewer.add_image(images, channel_axis = 1)

[<Image layer 'Image' at 0x7fa02aeb7d60>,
 <Image layer 'Image [1]' at 0x7fa03a8c5ca0>,
 <Image layer 'Image [2]' at 0x7fa03a2c6490>]

In [180]:
print()

# Load tracks

In [21]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/06/07 01:09:01 pm] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/06/07 01:09:01 pm] Loading tracks/obj_type_1
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading tracks/obj_type_1
[INFO][2023/06/07 01:09:01 pm] Loading LBEP/obj_type_1
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading LBEP/obj_type_1
[INFO][2023/06/07 01:09:01 pm] Loading objects/obj_type_1 (46507, 5) (46507 filtered: None)
07-Jun-23 13:09:01 - btrack.io.hdf - INFO     - Loading objects/obj_type_1 (46507, 5) (46507 filtered: None)
[INFO][2023/06/07 01:09:11 pm] Loading segmentation (75, 6048, 6048)
07-Jun-23 13:09:11 - btrack.io.hdf - INFO     - Loading segmentation (75, 6048, 6048)
[INFO][2023/06/07 01:09:11 pm] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5
07-Jun-23 13:09:11 - btrack.io.hdf - INFO     - Clo

In [59]:
filtered_tracks = [t for t in tracks if len(t) > 25]

In [61]:
napari_filtered_tracks, properties, graph = btrack.utils.tracks_to_napari(filtered_tracks, 
                                                                    ndim=2)

### Recolour tracks

In [41]:
col_segmentation = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

# Launch napari image viewer

In [93]:
%time
viewer = napari.Viewer(title = '3,10 sum projection')

# viewer.add_image(images, 
#                  channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
#                  visible = True
#                  )
viewer.add_image(images, 
                 channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )
# viewer.add_labels(segmentation, 
# #                   num_colors= 1,
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                   name = 'segmentation'
#                  )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,5.04,5.04)
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,5.04,5.04),
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 38.9 µs


[<Image layer 'Image' at 0x7f1fad5f1970>,
 <Image layer 'Image [1]' at 0x7f207d2ee880>,
 <Image layer 'Image [2]' at 0x7f1f924f0880>]

Rendering frames...


  4%|██                                                   | 4/100 [02:21<1:02:44, 39.22s/it]

In [25]:
visualise.highlight_cell(1251, viewer, tracks)

<Points layer 'cell 1251' at 0x7f235d482790>

In [62]:
viewer.add_tracks(napari_filtered_tracks, scale = (1,5.04,5.04),)

<Tracks layer 'napari_filtered_tracks [2]' at 0x7ff4e5a32430>

In [25]:
viewer.layers['napari_filtered_tracks'].scale = (30, 5.04, 5.04)

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 227, in <lambda>
    button.clicked.connect(lambda: self.trigger(name))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 427, in trigger
    return self._actions[name].injected()
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/in_n_out/_store.py", line 773, in _exec
    result = func(**{**_kwargs, **bound.arguments})
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/image/_image_key_bindings.py", line 33, in orient_plane_normal_along_x
    orient_plane_normal_around_cursor(layer, plane_normal=(0, 0, 1))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/utils/interactivity_utils.py", line 126, in orient_plane_normal_around_cursor
    view_direction = layer._world_to_displayed_data_ray(
  File "/home/dayn/minic

In [32]:
viewer.layers['napari_filtered_tracks'].axis_labels=["time", "y", "x"]

In [29]:
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
                    name="hacky downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (1,1,1)
                )
# viewer.add_labels(ds_segmentation, 
#                  name = 'downscaled segmentation')

viewer.add_labels(ds_col_segmentation, 
                  name = 'downscaled coloured segmentation',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

viewer.add_labels(ds_col_segmentation_new_tracks, 
                  name = 'downscaled coloured segmentation new tracks',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

<Labels layer 'downscaled coloured segmentation new tracks' at 0x7f15c273ca30>